
# Task 5: Auto Tagging Support Tickets Using LLM

## Objective
Automatically classify support tickets into categories using:
- Zero-shot learning
- Few-shot learning

Also output Top 3 probable tags per ticket.


In [1]:

# Install once if needed
!pip install transformers torch scikit-learn


In [2]:

import pandas as pd
from transformers import pipeline
from sklearn.metrics import classification_report



## Load Dataset


In [3]:

df = pd.read_csv("support_tickets.csv")
df.head()


,ticket_text,true_tag
0,My internet is not working since yesterday.,Technical Issue
1,I was charged twice for my subscription.,Billing
2,Unable to reset my password.,Account Access
3,The mobile app keeps crashing.,Technical Issue
4,I want to upgrade my current plan.,Sales



## Zero-Shot Classification
Using Hugging Face zero-shot classifier.


In [4]:

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["Billing", "Technical Issue", "Account Access", "Sales"]

def classify_ticket(text):
    result = classifier(text, candidate_labels)
    return result["labels"][:3]

df["predicted_top3"] = df["ticket_text"].apply(classify_ticket)

df.head()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

,ticket_text,true_tag,predicted_top3
0,My internet is not working since yesterday.,Technical Issue,"[Technical Issue, Account Access, Billing]"
1,I was charged twice for my subscription.,Billing,"[Billing, Account Access, Technical Issue]"
2,Unable to reset my password.,Account Access,"[Technical Issue, Account Access, Sales]"
3,The mobile app keeps crashing.,Technical Issue,"[Technical Issue, Billing, Account Access]"
4,I want to upgrade my current plan.,Sales,"[Technical Issue, Account Access, Billing]"



## Evaluation (Top-1 Only)


In [5]:

df["predicted_top1"] = df["predicted_top3"].apply(lambda x: x[0])

print(classification_report(df["true_tag"], df["predicted_top1"]))


                 precision    recall  f1-score   support

 Account Access       1.00      0.33      0.50         3
        Billing       1.00      1.00      1.00         3
          Sales       0.00      0.00      0.00         1
Technical Issue       0.50      1.00      0.67         3

       accuracy                           0.70        10
      macro avg       0.62      0.58      0.54        10
   weighted avg       0.75      0.70      0.65        10



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



## Few-Shot Prompting Example


In [6]:

def few_shot_classify(text):
    prompt = f"""
You are a support ticket classifier.
Categories: Billing, Technical Issue, Account Access, Sales.

Examples:
Ticket: I was charged twice.
Category: Billing

Ticket: Cannot login to my account.
Category: Account Access

Now classify:
Ticket: {text}
Category:
"""
    result = classifier(prompt, candidate_labels)
    return result["labels"][0]

df["few_shot_prediction"] = df["ticket_text"].apply(few_shot_classify)

df.head()


,ticket_text,true_tag,predicted_top3,predicted_top1,few_shot_prediction
0,My internet is not working since yesterday.,Technical Issue,"[Technical Issue, Account Access, Billing]",Technical Issue,Billing
1,I was charged twice for my subscription.,Billing,"[Billing, Account Access, Technical Issue]",Billing,Billing
2,Unable to reset my password.,Account Access,"[Technical Issue, Account Access, Sales]",Technical Issue,Billing
3,The mobile app keeps crashing.,Technical Issue,"[Technical Issue, Billing, Account Access]",Technical Issue,Billing
4,I want to upgrade my current plan.,Sales,"[Technical Issue, Account Access, Billing]",Technical Issue,Billing



## Final Summary

- Generated synthetic support ticket dataset
- Implemented zero-shot classification
- Implemented few-shot prompting
- Evaluated performance
- Returned Top 3 probable tags

This completes Task 5.
